In [74]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as 
import math

read the data in from txt file

In [112]:
with open('input.txt', 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines] # remove new line char
lines = [line for line in lines if line != ""] # remove empties
lines = list(map(str.lower, lines)) # make everything lower to reduce vocab

In [119]:
# now let's make everything one long string

data = ' '.join(lines)
chars = sorted(set(data))

print(len(data))



1108166


In [120]:
tokenize = {char : index for index, char in enumerate(chars) }
detokenize = {char : index for char, index in enumerate (list(tokenize.keys()))}

In [123]:
tokens = []

for char in data:
    tokens.append(tokenize[char])
    

In [131]:
num_embeddings = len(tokenize.keys())
embedding_dim = 50

In [133]:
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)


    def forward(self, x):
        x = self.embedding(x)

        return x
        
        

In [135]:
model = Model(num_embeddings, embedding_dim)

In [136]:
# testing model

input_indices = torch.tensor([12, 15, 23, 1], dtype=torch.long)


In [137]:
input_indices

tensor([12, 15, 23,  1])

In [138]:
# Evaluate model
model.eval()
output = model(input_indices)

# Print output embeddings
print("Output Embeddings:")
print(output)

Output Embeddings:
tensor([[-4.5354e-01, -9.8725e-01, -6.0165e-01,  7.4291e-01, -8.3635e-01,
         -2.5668e-01,  5.6757e-01,  4.6428e-01,  2.3700e-01, -2.9635e-02,
          5.1507e-01, -3.6878e-01, -1.8067e+00, -1.0763e+00,  1.7325e+00,
         -6.1952e-01,  3.9473e-01,  6.9312e-01,  6.3261e-01, -6.2907e-01,
          2.2160e+00,  6.6611e-01, -1.2564e+00,  3.2911e-01, -5.8157e-01,
         -3.2110e-01,  1.0831e+00, -3.7754e-01, -8.5314e-01,  2.9414e-01,
         -3.0067e-01, -2.0369e+00,  1.3080e+00,  1.0431e+00,  1.8659e+00,
          1.5184e+00, -1.8173e+00,  2.2482e+00,  1.7383e-01,  1.3350e-01,
          2.0587e-01,  8.7298e-01, -6.2319e-01,  1.3300e+00,  3.7248e-01,
          2.5250e-01, -6.3392e-01, -1.2900e+00, -1.2507e+00, -3.6913e-01],
        [ 6.6035e-01,  3.5890e-01,  9.9214e-01, -8.8784e-01,  1.0917e+00,
          8.5483e-01, -7.5907e-01, -1.8464e-01,  1.3000e+00, -9.0745e-01,
         -1.7746e-01,  1.1715e-01,  6.8623e-01,  4.2686e-01,  1.4034e+00,
          2.2308e-

In [139]:
output.shape

torch.Size([4, 50])